# Installing Dependencies

In [ ]:
%%capture
!pip install langchain

In [ ]:
%%capture
!pip install openai unstructured chromadb

In [ ]:

!pip install openai
!pip install unstructured
!pip install python-magic
!pip install chromadb
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.9/index.html
!pip install layoutparser
!pip install pillow


In [ ]:
%%capture
!apt-get install -y libmagic-dev
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr

In [ ]:
%%capture
!pip install tiktoken

In [ ]:
%%capture
!pip install azure-storage-blob

In [ ]:
!pip install --force-reinstall Pillow


  Using cached Pillow-9.5.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.4 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0


In [ ]:
!apt-get install -y libreoffice
!soffice --version


# Importing Dependencies

In [ ]:
import langchain
import openai
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain import OpenAI, VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
# import magic
import nltk
nltk.download('punkt')

os.environ["OPENAI_API_KEY"] = ""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Define the directory path
# directory_path = '/content/drive/MyDrive/FAQ/FAQ'

# from langchain.document_loaders import DirectoryLoader, TextLoader, PDFLoader, CSVLoader, PPTLoader

# # Initialize the DirectoryLoader
# loader = DirectoryLoader(directory_path, show_progress=True)

# # Load the documents from the directory
# documents = []
# for file_path in loader.iterate_files():
#     if file_path.endswith('.txt'):
#         text_loader = TextLoader(file_path)
#         text_document = text_loader.load()
#         documents.append(text_document)
#     elif file_path.endswith('.pdf'):
#         pdf_loader = PDFLoader(file_path)
#         pdf_document = pdf_loader.load()
#         documents.append(pdf_document)
#     elif file_path.endswith('.csv'):
#         csv_loader = CSVLoader(file_path)
#         csv_document = csv_loader.load()
#         documents.append(csv_document)
#     elif file_path.endswith('.ppt'):
#         ppt_loader = PPTLoader(file_path)
#         ppt_document = ppt_loader.load()
#         documents.append(ppt_document)

# print(f"You have {len(documents)} document(s) in your data")

# # Further processing of the documents...



# Loading Documents from Blob Storage

In [ ]:
from langchain.document_loaders import AzureBlobStorageContainerLoader
loader = AzureBlobStorageContainerLoader(conn_str="DefaultEndpointsProtocol=https;AccountName=openaiblob1;EndpointSuffix=core.windows.net ",container="openaicontainer1")
document = loader.load()

print (f'You have {len(document)} document(s) in your data')


# Text Splitting
## ( chunk size= 200 , chunk overlap= 40)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=40,
)

documents = text_splitter.split_documents(docs)
documents[:10]

# Embeddings

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
API_KEY= os.environ["OPENAI_API_KEY"] = ""
embeddings = OpenAIEmbeddings(openai_api_key=API_KEY)

# Vectore Store Creation

In [ ]:

import pickle

# Create ChromaDB Vector Store from documents and embeddings
vectstore = Chroma.from_documents(documents, embeddings)


qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=vectstore)

query = "Write abput different lenses in light"
qa.run(query)


/usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:206: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


' Lenses are curved pieces of glass that can be used to bend and focus light. Lenses are used in many optical instruments, such as microscopes, telescopes, binoculars, cameras, and other optical devices. They are also used in eyeglasses and contact lenses to help people see better.'

In [ ]:
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents, embeddings)
query = "employee data on Adeline Huang?"
docs = vectorstore.similarity_search(query)

# Querying

In [ ]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=vectstore, return_source_documents=True)
query = "Write a summary of football"
result = qa({'query': query})


In [ ]:
result['result']


" The football write-up covers the exhilarating match between two rival teams, showcasing incredible skills, intense competition, and a nail-biting finish. It highlights the star player's outstanding performance, the team's tactical brilliance, and the roaring crowd's electrifying atmosphere. The write-up captures the essence of the game in just 40 words."

In [ ]:
# Continue the conversation
response = qa({"query": "And how is their habitat?"})
print(response)

{'query': 'And how is their habitat?', 'result': " Fuzzle's natural habitat is generally found in woodlands, where there is plenty of food and opportunities for exploration and socializing. These habitats are typically well-protected from the elements and provide plenty of shelter and safety for Fuzzles to thrive."}


In [ ]:
from langchain.document_loaders import AzureBlobStorageContainerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain import RetrievalQA

import pickle

# Load documents from Azure Blob Storage container
loader = AzureBlobStorageContainerLoader(
    conn_str="DefaultEndpointsProtocol=https;AccountName=translatorstore;EndpointSuffix=core.windows.net",
    container="translator-container1"
)
docs = loader.load()

print(f"You have {len(docs)} document(s) in your data")

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50
)
documents = text_splitter.split_documents(docs)

# Initialize OpenAI Embeddings
API_KEY = ""
embeddings = OpenAIEmbeddings(openai_api_key=API_KEY, model='text-embedding-ada-002')

# Create ChromaDB Vector Store from documents and embeddings
vectorstore = Chroma.from_documents(documents, embeddings)

# # Store the vector store to a file
# vectorstore_file = "vectorstore.pkl"
# with open(vectorstore_file, "wb") as f:
#     pickle.dump(vectorstore, f)
# print(f"Vector store is stored in {vectorstore_file}")

# Initialize the LLM and QA model
llm = OpenAI(openai_api_key=API_KEY, model_name="text-davinci-003",
              temperature=0,
              max_tokens=300, # Maximal number of tokens returned by the LLM
              chain_type="stuff" )

qa = RetrievalQA.from_chain_type(llm=OpenAIChat(model='gpt-3.5-turbo'), chain_type='stuff', retriever=vector_store.as_retriever())


# # Initialize the MemoryAgent
# memory_agent = MemoryAgent()

# Previous conversation memory
previous_conversation = []

# User input and conversation loop
print("Assistant: Hello! How can I assist you today?")
while True:
    query = input("User: ")

    # Add user query to the conversation
    previous_conversation.append({"role": "user", "message": query})

    # Retrieve response from the QA model
    response = qa.run(query)

    # Add QA response to the conversation
    previous_conversation.append({"role": "assistant", "message": response})

    # Retrieve previous user queries and responses from the conversation
    previous_queries = [item["message"] for item in previous_conversation if item["role"] == "user"]
    previous_responses = [item["message"] for item in previous_conversation if item["role"] == "assistant"]

    # Print previous conversation
    print("Previous Conversation:")
    for user_query, assistant_response in zip(previous_queries, previous_responses):
        print("User:", user_query)
        print("Assistant:", assistant_response)
        print()

    print("Assistant:", response)


ModuleNotFoundError: ignored

In [ ]:

   #load the faiss vector store we saved into memory
    vectorStore = FAISS.load_local("./dbs/documentation/faiss_index", embeddings)

    #use the faiss vector store we saved to search the local document
    retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k":2})

    #use the vector store as a retriever
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=False)
    while True:
        query = input('you: ')
        if query == 'q':
            break
        ask_question(qa, query)